In [5]:
import zipfile
import hashlib
import csv
import os

In [1]:
# jar 파일을 압축을 해지합니다.
jarPaths = []

for i in range(2):
    inputDir = input("jar 파일 경로와 이름을 입력하세요. : ")
    jarPaths.append(inputDir)

jar 파일 경로와 이름을 입력하세요. : \\192.168.2.21\Package\빌드 형상 관리\MASR CDR\v5.0.1\cdr-5.0.1.1.43.slcp
jar 파일 경로와 이름을 입력하세요. : \\192.168.2.21\Package\빌드 형상 관리\MASR CDR\v5.0.1\cdr-5.0.1.2.44.slcp


In [7]:
password = b"b002cd031989a8118aab022d74616efd5718f399"

In [2]:
# Function to compute the hash of file contents
def compute_hash(file, algorithm="sha256"):
    hash_func = hashlib.new(algorithm)
    hash_func.update(file.read())
    return hash_func.hexdigest()

In [15]:
# 압축 파일내에 HASH 값을 계산하고 CSV 파일로 저장합니다. 
def get_file_hashes(jar_path):
    file_hashes = {}
    filePath, fileName = os.path.split(jar_path)
        
    with open(filePath + "\\" + fileName + ".csv", mode = "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['jar File', 'File Name', 'Hash Value'])
        
        with zipfile.ZipFile(jar_path, 'r') as package:
            for file_name in package.namelist():
                if not file_name.endswith("/"):  # Skip directories
                    with package.open(file_name, pwd=password) as file:
                        # Compute and store the hash
                        print(file_name)
                        file_hashes[file_name] = compute_hash(file)
                        writer.writerow([jarPaths[i], file_name, file_hashes[file_name]])
                                
    return file_hashes

In [16]:
# Get file hashes from both .jar files
hashes_jar1 = get_file_hashes(jarPaths[0])
hashes_jar2 = get_file_hashes(jarPaths[1])

cdr-5.0.1.1.jar
cdr-5.0.1.2.jar


In [10]:
outputCsv = input("결과 csv 저장할 경로와 파일명을 입력하세요.")

결과 csv 저장할 경로와 파일명을 입력하세요.D:\04_Test\CDR\v5.0.1.2_AB_Test.csv


In [11]:
# Compare the hashes of files in both .jar files
with open(outputCsv, "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['file_name', jarPaths[0], jarPaths[1]])
    
    for file_name in hashes_jar1:
        if file_name in hashes_jar2:
            if hashes_jar1[file_name] != hashes_jar2[file_name]:
                writer.writerow([file_name, hashes_jar1[file_name], hashes_jar2[file_name]])
                # Print file name and hash values if they differ
                print(f"File '{file_name}' differs between the jars.")
                print(f"  Hash in {jarPaths[0]}: {hashes_jar1[file_name]}")
                print(f"  Hash in {jarPaths[1]}: {hashes_jar2[file_name]}")
        else:
            # Print file name and hash value if it's only in the first jar
            writer.writerow([file_name, hashes_jar1[file_name], "-"])
            print(f"File '{file_name}' is only present in {jarPaths[0]}.")
            print(f"  Hash: {hashes_jar1[file_name]}")

    # Check for files in the second jar that are not in the first jar
    for file_name in hashes_jar2:
        if file_name not in hashes_jar1:
            # Print file name and hash value if it's only in the second jar
            writer.writerow([file_name, "-", hashes_jar2[file_name]])
            print(f"File '{file_name}' is only present in {jarPaths[1]}.")
            print(f"  Hash: {hashes_jar2[file_name]}")


File 'cdr-5.0.1.1.jar' is only present in \\192.168.2.21\Package\빌드 형상 관리\MASR CDR\v5.0.1\cdr-5.0.1.1.43.slcp.
  Hash: 92fd9e24eb82f99b435ebf256badcafa2b6f48a162ccf63b447020574142b9e4
File 'cdr-5.0.1.2.jar' is only present in \\192.168.2.21\Package\빌드 형상 관리\MASR CDR\v5.0.1\cdr-5.0.1.2.44.slcp.
  Hash: 053179b0734906d3609d6125ff2bc1d01c27726dca2076ac1196b65b58607892


In [14]:
# CSV 파일에 대상, 내부 파일명, Hash 값 저장하기
for i in range(2):
    filePath, fileName = os.path.split(jarPaths[i])
    
    with open(filePath + "\\" + fileName + ".csv", mode = "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        writer.writerow(['jar File', 'File Name', 'Hash Value'])
        
        with zipfile.ZipFile(jarPaths[i], 'r') as jar:
            for file_name in jar.namelist():
                if not file_name.endswith("/"):
                      with jar.open(file_name) as file:
                            file_hash = compute_hash(file, algorithm="sha256")
                            writer.writerow([jarPaths[i], file_name, file_hash])

TypeError: __init__() got an unexpected keyword argument 'pwd'

NameError: name 'hashes_jar1' is not defined